# 💾 04. Data Persistence

## 📑 Meta-Intro
- **Key Topics:** The Ephemeral Filesystem, Named Volumes (`-v`), Bind Mounts, and `tmpfs`.
- **Executive Summary:** By default, containers are **stateless**. If you delete a database container, all your user data is deleted with it. This behavior is intentional but problematic for databases and persistent logs. In this notebook, we will learn the two primary ways to persist data: **Named Volumes** (for production databases) and **Bind Mounts** (for development code syncing).

---

## 1. 📉 The Problem: Ephemeral Filesystem

When a container is started, it uses a writable layer on top of the image's read-only layers. When the container is removed (`docker rm`), this writable layer is destroyed.

**The Rule:** Never store important data (Customer records, Logs, Code) inside the container's writable layer.

---

## 2. 🗄️ Named Volumes (The "Production" Way)

**Named Volumes** are storage areas fully managed by Docker. They live in a protected area of the host filesystem (usually `/var/lib/docker/volumes` on Linux) and should not be modified by non-Docker processes.

### Characteristics:
* **Managed:** Docker handles permissions and ownership.
* **Persistent:** They survive container deletion.
* **Sharable:** Multiple containers can attach to the same volume.

### Usage:
```bash
# Create explicitly (Optional, Docker creates it automatically if missing)
docker volume create my-db-data

# Mount it to the container
# Syntax: -v <volume_name>:<container_path>
docker run -d -v my-db-data:/var/lib/postgresql/data postgres
```



## 3. 🔗 Bind Mounts (The "Dev" Way)

**Bind Mounts** map a specific file or directory on your **Host Machine** (e.g., your laptop's project folder) into the container.

### Characteristics:
* **Direct Access:** You can edit code in VS Code on Windows, and the changes appear instantly inside the Linux container.
* **Host Dependent:** Relies on the host's directory structure (paths differ between Mac/Windows/Linux).
* **Performance:** Slightly slower than Volumes due to filesystem translation (especially on Windows/Mac).

### Usage:
```bash
# Mount the current directory ($(pwd)) to /app inside the container
# Syntax: -v <host_path>:<container_path>
docker run -v $(pwd):/app node:18
```

---

## 4. ⚔️ Comparison: Volumes vs. Bind Mounts

| Feature | Named Volumes | Bind Mounts |
| :--- | :--- | :--- |
| **Location** | Docker-managed (`/var/lib/docker/...`) | Any path on Host |
| **Use Case** | Databases, Data persistence | Live Code Reloading, Config injection |
| **Portability** | High (Works on any Docker host) | Low (Path dependent) |
| **CLI Syntax** | `-v name:/path` | `-v /host/path:/path` |

---

## 🌊 Mini-Challenge: The Immortal Database

**Goal:** Prove that data persists even after a database container is destroyed.

**Instructions:**
1.  Run a Redis container named `redis-1` with a volume named `my-redis-data` mapped to `/data`.
2.  Connect to it and write a key: `docker exec redis-1 redis-cli set mykey "Hello Persistence"`.
3.  **Destroy** the container: `docker rm -f redis-1`.
4.  Run a **new** container `redis-2` attached to the **same volume**.
5.  Verify the data exists: `docker exec redis-2 redis-cli get mykey`.
6.  **Cleanup:** Remove container and volume (`docker volume rm`).

---

In [ ]:
%%bash
# SOLUTION REFERENCE

# 1. Run Container 1 with Volume
docker run -d --name redis-1 -v my-redis-data:/data redis

# 2. Write Data
docker exec redis-1 redis-cli set mykey "Hello Persistence"

# 3. Destroy Container 1 (Data would be lost here if not for the volume)
docker rm -f redis-1

# 4. Run Container 2 with SAME Volume
docker run -d --name redis-2 -v my-redis-data:/data redis

# 5. Verify Data
docker exec redis-2 redis-cli get mykey

# 6. Cleanup
docker rm -f redis-2
docker volume rm my-redis-data

## 🎓 Core Insight for Your CSE Career

**Separation of Compute and State**

In cloud architecture, we treat **Compute** (the container) as disposable and **State** (the volume) as precious. This decoupling allows us to upgrade our database version simply by deleting the old Postgres 14 container and pointing a new Postgres 15 container at the same Volume (after ensuring compatibility, of course). This architectural pattern is fundamental to Kubernetes and Cloud Native design.
